In [1]:
import numpy as np; import pandas as pd
import re; import datetime; from collections import Counter, OrderedDict
import category_encoders as ce
from sklearn.model_selection import train_test_split
# import pylogit as pl

In [20]:
df = pd.read_csv(r'가마우지\race_result.csv', engine='python', encoding='utf-8')

In [25]:
df.columns

Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1',
       'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m', 'weight_diff',
       'raw_weight', 'weight_added', 'distance', 'velocity', 'prev1_rank',
       'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo', 'prev3_velo',
       'horse_level_num', 'year', 'month', 'choice', 'choice2', 'choice3',
       'danwin1yr', 'bokwin1yr', 'samwin1yr', 'jdanwin1yr', 'jbokwin1yr',
       'jsamwin1yr', 'jprev1_rank', 'jprev2_rank', 'jprev3_rank', 'humidity',
       'race_level', 'new_distance'],
      dtype='object')

In [26]:
df['groupid'] = df['date'] + df['round'].apply(str)

In [27]:
idset = list(set(df['groupid']))
give_ids = dict(list(zip(idset, list(range(len(idset))))))

In [28]:
df['groupid'] = df['groupid'].apply(lambda x: give_ids[x])
df = df[df['distance'] > 0] # 거리 정보 있는 것만 살림

In [29]:
distances = list(set(df['distance']))
avg_vel = {}

In [30]:
for i in distances:
    avg_vel[i] = df[df['distance'] == i]['velocity'].mean()

In [31]:
df['avg_vel'] = df['distance'].apply(lambda x: avg_vel[x])
df['adj_vel'] = df['velocity'] / df['avg_vel']

In [32]:
df['prev1_rank']=df['prev1_rank'].fillna('0'); df['prev2_rank']=df['prev2_rank'].fillna('0')
df['jprev1_rank']=df['jprev1_rank'].fillna('0'); df['jprev2_rank']=df['jprev2_rank'].fillna('0')

In [36]:
df = df[(df['prev1_rank'] != '0') & (df['prev2_rank'] != '0') & (df['jprev1_rank'] != '0') & (df['jprev2_rank'] != '0')]
#df = df[(df['prev1_rank'] != '0') & (df['prev2_rank'] != '0') & (df['jprev1_rank'] != '0') & (df['jprev2_rank'] != '0')]

In [38]:
set(df['prev2_rank'])

{'1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '실',
 '제',
 '중',
 '취'}

In [41]:
nums = list(map(str,range(1,17)))
def return_rank(x):
    if x in nums: return int(x)
    else: return 0
    
df['rank'] = df['rank'].apply(return_rank)
df['prev1_rank'] = df['prev1_rank'].apply(return_rank)
df['prev2_rank'] = df['prev2_rank'].apply(return_rank)
df['jprev1_rank'] = df['jprev1_rank'].apply(return_rank)
df['jprev2_rank'] = df['jprev2_rank'].apply(return_rank)
# df = df[(df['prev1_rank'].apply(lambda x: x in nums)) & (df['prev2_rank'].apply(lambda x: x in nums))]
# df['prev1_rank'] = df['prev1_rank'].apply(int)
# df['prev2_rank'] = df['prev2_rank'].apply(int)

In [44]:
for i in df.index:
    a = df.loc[i, 'prev1_rank']; b = df.loc[i, 'prev2_rank']
    if a != 0 and b != 0:
        df.loc[i, 'prev_rank'] = (a+b)/2
    elif a == 0 and b == 0:
        df.loc[i, 'prev_rank'] = 0
        df.loc[i, 'false2time'] = 1
    else:
        df.loc[i, 'prev_rank'] = a+b
        df.loc[i, 'false1time'] = 1
#del df['prev1_rank'], df['prev2_rank']

In [45]:
df['false1time'] = df['false1time'].fillna(0)
df['false2time'] = df['false2time'].fillna(0)

In [46]:
for i in df.index:
    a = df.loc[i, 'jprev1_rank']; b = df.loc[i, 'jprev2_rank']
    if a != 0 and b != 0:
        df.loc[i, 'jprev_rank'] = (a+b)/2
    elif a == 0 and b == 0:
        df.loc[i, 'jprev_rank'] = 0
        df.loc[i, 'jfalse2time'] = 1
    else:
        df.loc[i, 'jprev_rank'] = a+b
        df.loc[i, 'jfalse1time'] = 1
        
df['jfalse1time'] = df['jfalse1time'].fillna(0)
df['jfalse2time'] = df['jfalse2time'].fillna(0)
#del df['prev1_rank'], df['prev2_rank']

In [48]:
df['norating'] = (df['horse_level_num'] == 10).apply(int)

In [49]:
df['sex1'] = (df['sex'] == '수').apply(int)
df['sex2'] = (df['sex'] == '암').apply(int)

In [58]:
df.columns

Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1',
       'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m', 'weight_diff',
       'raw_weight', 'weight_added', 'distance', 'velocity', 'prev1_rank',
       'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo', 'prev3_velo',
       'horse_level_num', 'year', 'month', 'choice', 'choice2', 'choice3',
       'danwin1yr', 'bokwin1yr', 'samwin1yr', 'jdanwin1yr', 'jbokwin1yr',
       'jsamwin1yr', 'jprev1_rank', 'jprev2_rank', 'jprev3_rank', 'humidity',
       'race_level', 'new_distance', 'groupid', 'avg_vel', 'adj_vel',
       'prev_rank', 'false1time', 'false2time', 'jprev_rank', 'jfalse1time',
       'jfalse2time', 'norating', 'sex1', 'sex2'],
      dtype='object')

In [60]:
df.to_csv('hyunse_df.csv', encoding = 'utf-8')

In [71]:
df = pd.read_csv('hyunse_df.csv', encoding = 'utf-8')

In [75]:
groupids = list(set(df['groupid']))
train, test = train_test_split(groupids, test_size = 0.3, random_state=datetime.datetime.now().second)

train = df[df['groupid'].apply(lambda x: x in train)]
test = df[df['groupid'].apply(lambda x: x in test)]

In [76]:
train2 = train[train['rank']!=1]
train2['choice'] = (train2['rank'] == 2).apply(int)
train2['groupid'] = train2['groupid'] + 10000

train3 = train[train['rank']!=2]
train3['choice'] = (train3['rank'] == 3).apply(int)
train3['groupid'] = train3['groupid'] + 20000

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [77]:
train = train.append(train2, ignore_index=True).append(train3, ignore_index=True)

In [78]:
del train2, train3

In [79]:
train.columns

Index(['url', 'date', 'round', 'name', 'code', 'rank', 'lane', 'sex', 'age',
       'jockey_w', 'rating', 'jockey', 'difference', 'weight', 'dandivi',
       'yeondivi', 's1fr', 'c1r', 'c2r', 'c3r', 'c4r', 'g1fr', 's1f', 'c1',
       'c2', 'c3', 'c4', 'g3f', 'g1f', 'record', 'cure_in_1m', 'weight_diff',
       'raw_weight', 'weight_added', 'distance', 'velocity', 'prev1_rank',
       'prev2_rank', 'prev3_rank', 'prev1_velo', 'prev2_velo', 'prev3_velo',
       'horse_level_num', 'year', 'month', 'choice', 'choice2', 'choice3',
       'danwin1yr', 'bokwin1yr', 'samwin1yr', 'jdanwin1yr', 'jbokwin1yr',
       'jsamwin1yr', 'jprev1_rank', 'jprev2_rank', 'jprev3_rank', 'humidity',
       'race_level', 'new_distance', 'groupid', 'avg_vel', 'adj_vel',
       'prev_rank', 'false1time', 'false2time', 'jprev_rank', 'jfalse1time',
       'jfalse2time', 'norating', 'sex1', 'sex2'],
      dtype='object')

In [80]:
use = ['choice', 'groupid', 'lane', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'weight_diff', 'raw_weight', 'horse_level_num',\
      'new_distance', 'adj_vel', 'prev_rank', 'false1time', 'false2time', 'jprev_rank', 'jfalse1time', 'jfalse2time', 'norating', 'sex1', 'sex2']

basic_specification = OrderedDict()
basic_names = OrderedDict()
li = list(range(1,17))
xs = ['lane', 'sex1', 'sex2', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'raw_weight', 'weight_added', 'horse_level_num', 'prev_rank', 'norating']
basic_specification["intercept"] = [li]
basic_names['intercept'] = ['annoying']

for i in xs:
    basic_specification[i] = [li]
    basic_names[i] = [i]

custom_alt_id = "lane"
obs_id_column = "groupid"
choice_column = "choice"

clogit_mnl = pl.create_choice_model(data=train_new.sort_values(by=['groupid']),
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

clogit_mnl.fit_mle(np.ones(14))
clogit_mnl.get_statsmodels_summary()

In [175]:
groupids = list(set(train['groupid']))

import random
tmps = []; memory = []

for i in groupids:
    dta = train[train['groupid']==i]
    diff = list(dta['lane'])
    dta = dta[dta['choice'] == 0]
    if len(dta)==0: 
        memory.append(i)
        continue
    for j in li:
        if j not in diff:
            r = random.randint(0, len(dta)-1)
            tmp=list(dta.iloc[r,:])
            tmp[0]=j
            tmps.append(tmp)

train_new = train.append(pd.DataFrame(tmps, columns = train.columns), ignore_index = True)

train_new = train_new[train_new['식별'] == False]

train_new['식별'] = train_new['groupid'].apply(lambda x: x in memory)

train_new

In [81]:
train = train[use].sort_values(by=['groupid'])

In [82]:
test = test[use].sort_values(by=['groupid'])

In [83]:
train.columns

Index(['choice', 'groupid', 'lane', 'age', 'jockey_w', 'dandivi', 'yeondivi',
       'cure_in_1m', 'weight_diff', 'raw_weight', 'horse_level_num',
       'new_distance', 'adj_vel', 'prev_rank', 'false1time', 'false2time',
       'jprev_rank', 'jfalse1time', 'jfalse2time', 'norating', 'sex1', 'sex2'],
      dtype='object')

In [84]:
train.to_csv('clogit_train2.csv', encoding='utf-8', index=False)
#train_new.to_csv('clogit_train_new.csv', encoding='utf-8', index=False)
test.to_csv('clogit_test2.csv', encoding='utf-8', index=False)

# STATA 코드들
- train data만 넣은 뒤 -
clogit var1 var2 var3 var4 var5 var6 var7 var8 var9 var10 var11 var12 var14 var15, group(var16)
- test data append 하고 -
predict phat
- 이후 csv로 test data 부분의 phat column만 export한다 -> 빈 칸들 100으로 대체해줌.

In [88]:
test = pd.read_csv('clogit_test2.csv', encoding = 'utf-8')

In [89]:
test = test.set_index(['groupid', 'lane'])

In [90]:
df = pd.read_csv('hyunse_df.csv', encoding = 'utf-8') # rank살리기...

In [91]:
# test data choice2, 3 살리기..
df = df.set_index(['groupid', 'lane'])

In [92]:
test['rank'] = df['rank']

In [94]:
test['choice2'] = test['choice'] + (test['rank'] == 2)
test['choice3'] = test['choice2'] + (test['rank'] == 3)

In [95]:
test = test.reset_index()

In [105]:
why=list(pd.read_csv('dfdsaf.csv').iloc[:,0]) # STATA로 뽑은 phat

In [106]:
test['phat'] = why

In [107]:
test['delete']=test['phat'].apply(lambda x: x == 100)
test = test[test['delete'] == 0]

In [108]:
test['phat']=test['phat'].apply(float)

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [109]:
idx = list(set(test['groupid']))
dictt=dict(test.groupby(['groupid'])['phat'].max())
dandivi = []

t1=0; t2=0; t3=0
for i in idx:
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice'])[0] == 1:
        t1 = t1 + 1
        #dandivi.append(list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['dandivi'])[0])
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice2'])[0] == 1:
        t2 = t2 + 1
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice3'])[0] == 1:
        t3 = t3 + 1 

In [85]:
import numpy as np
(np.array(dandivi) >= 3).mean()

0.292894280762565

In [110]:
total = test.groupby(['groupid']).max()['choice'].sum()
total2 = test.groupby(['groupid']).max()['choice2'].sum()
total3 = test.groupby(['groupid']).max()['choice3'].sum()

In [111]:
print(t1 / total)
print(t2 / total2)
print(t3 / total3)

0.5952685421994884
0.7314652656158785
0.781786941580756


In [112]:
for_accu=dict(test.groupby(['groupid']).count()['lane']<5)

In [113]:
test['foraccu'] = test['groupid'].apply(lambda x: for_accu[x])

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [114]:
test_n = test[test['foraccu']==False]

In [115]:
set(test_n['rank'])

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [126]:
idx = list(set(test_n['groupid']))
dictt=dict(test_n.groupby(['groupid'])['phat'].max())
dandivi = []

t1=0; t2=0; t3=0
for i in idx:
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice'])[0] == 1:
        t1 = t1 + 1
        #dandivi.append(list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['dandivi'])[0])
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice2'])[0] == 1:
        t2 = t2 + 1
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice3'])[0] == 1:
        t3 = t3 + 1 

In [120]:
total = test_n.groupby(['groupid']).max()['choice'].sum()
total2 = test_n.groupby(['groupid']).max()['choice2'].sum()
total3 = test_n.groupby(['groupid']).max()['choice3'].sum()

In [116]:
idx = list(set(test_n['groupid'])) # n번째로 확률 높은 애가 몇 등할지?
#dictt=dict(test_n.groupby(['groupid'])['phat'].max())
dandivi = []

t1n=0; t2n=0; t3n=0
cnt=0

for i in idx:
    rnk = list(test_n[test_n['groupid']==i]['rank'])
    tmp = test_n[test_n['groupid']==i].sort_values(by=['phat'], ascending=False).iloc[0,:] # iloc index 바꾸면 몇 등인지 뺄 수 있겠음.
    tmp2 = test_n[test_n['groupid']==i].sort_values(by=['phat'], ascending=False).iloc[1,:]
#     print(tmp['phat'])
#     break
    if 1 in rnk and 2 in rnk: cnt = cnt + 1
    if tmp['rank'] == 1 and tmp2['rank'] == 2:
        t1n = t1n + 1
        #dandivi.append(list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['dandivi'])[0])
#     if tmp['choice2'] == 1:
#         t2n = t2n + 1
#     if tmp['choice3'] == 1:
#         t3n = t3n + 1 

# totaln = test_n.groupby(['groupid']).max()['choice'].sum()
# total2n = test_n.groupby(['groupid']).max()['choice2'].sum()
# total3n = test_n.groupby(['groupid']).max()['choice3'].sum()

In [117]:
t1n / cnt

0.3361462728551336

In [127]:
print(t1 / total)
print(t2 / total2)
print(t3 / total3)

0.5875816993464053
0.7251356238698011
0.7771156138259833
